In [1]:
import pandas as pd
import numpy as np
import config
from db_funcs import dropTable, get_mssql_connection
from weborama_parse_email_report import parse_type_category, get_product_from_insertion
import os
from datetime import date
from datetime import datetime
import warnings 
db_name = config.db_name

# file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\38_x5_report_parser'
# file_name = 'test.xlsx'

# df = pd.read_excel(os.path.join(file_path, file_name))

In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [3]:
conn = get_mssql_connection(db_name)
cursor = conn.cursor()


query = f'select distinct insertion from weborama_report_table'

warnings.simplefilter(action='ignore', category=UserWarning)
df = pd.read_sql(query, conn)

Все ок. Подключились!


In [4]:
df.head(2)

,insertion
0,byyd_|format_olv|type_rewarded-video|aud_progr_igronik_media_ge_kitaiskaya-eda_02_2025
1,yandex_|format_olv|type_videobanner|aud_cenit_igronik_media_cvp1_apelsin_02_2025


In [ ]:
df.shape

In [5]:
df['type'] = df['insertion'].apply(parse_type_category, flag='type_')
# парсим название Категории
df['category'] = df['insertion'].apply(parse_type_category, flag='format_')
# парсим название Продукта
df['product'] = df['insertion'].apply(get_product_from_insertion)

In [6]:
df['category'].drop_duplicates()

0       olv
2    banner
Name: category, dtype: object

In [7]:
df['type'].drop_duplicates()

0      rewarded-video
1         videobanner
2           universal
3        cross-stream
4              banner
6             premium
8           in-stream
11               main
14          multiroll
28        promobanner
49         fullscreen
133             bonus
Name: type, dtype: object

In [ ]:
flag = 'aud_progr_'#'type_'
start_index = row.find(flag)
cut_row = row[start_index+len(flag):]
end_index = cut_row.find('|')

In [ ]:
cut_row[:end_index] if end_index!=-1 else cut_row

In [ ]:
end_index

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df = df.sort_values(['date', 'category'])
df = df.reset_index(drop='True')

In [ ]:
df['first_not_blank'] = df[df['amount'] > 0].groupby('category')['date'].transform('min')
df['first_not_blank'] = df['first_not_blank'].fillna('0')

In [ ]:
df

In [ ]:
# df['amount_prev'] = np.where((df['first_not_blank']!=0)&(df['date']==df['first_not_blank']), 
#                            df['amount'], df.groupby('category')['amount'].shift(1))

In [ ]:
# df['amount_prev'] = df['amount_prev'].fillna(0)

In [ ]:
df['amount_not_blank'] = df.groupby(['category'])['amount'].cummax()

In [ ]:
df

In [ ]:
df['increment'] = np.where((df['first_not_blank']!=0)&(df['date']==df['first_not_blank']), 
                           df['amount_not_blank'], df['amount_not_blank'] - df.groupby('category')['amount_not_blank'].shift(1))

In [ ]:
df['increment'] = df['increment'].fillna(0)

In [ ]:
df

In [ ]:
# df['test'] = df.query('amount > 0').groupby('category')['date'].transform('min')

In [ ]:
# df['date'] = pd.to_datetime(df['date'])
# df['start_date'] = df.groupby('category')['date'].transform('min')

In [ ]:
df

In [ ]:
df.groupby('category')['increment'].sum()